# Setup


In [ ]:
import ee
ee.Authenticate(force=True)
ee.Initialize(project='ee-mrk2152')
import pandas as pd
import altair as alt
import numpy as np

# Code

In [ ]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):


  def reduce_region_function(img):

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

def filter_summer_months(df):
    return df[df['Month'].isin([6, 7, 8])]

In [ ]:
dcp_col = (ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS')
           .select(['tasmax_median', 'tasmin_median'])
           .filter(
               ee.Filter.And(ee.Filter.eq('scenario', 'rcp45'),
                             ee.Filter.date('2024-01-01', '2074-12-31'))))

def calc_mean_temp(img):
    return (img.select('tasmax_median')
            .add(img.select('tasmin_median'))
            .divide(ee.Image.constant(2.0))
            .rename(['Temp-mean'])
            .copyProperties(img, img.propertyNames()))

dcp_col = dcp_col.map(calc_mean_temp)

latitude = 36.144691
longitude = -114.417733
point = ee.Geometry.Point([longitude, latitude])

reduce_dcp30 = create_reduce_region_function(
    geometry=point, reducer=ee.Reducer.first(), scale=1000, crs='EPSG:4326')

dcp_stat_fc = ee.FeatureCollection(dcp_col.map(reduce_dcp30)).filter(
    ee.Filter.notNull(dcp_col.first().bandNames()))

dcp_dict = fc_to_dict(dcp_stat_fc).getInfo()
dcp_df = pd.DataFrame(dcp_dict)

dcp_df = add_date_info(dcp_df)

dcp_df['Temp-mean'] = dcp_df['Temp-mean'] - 273.15
dcp_df['Model'] = 'NEX-DCP30'

prism_col = (ee.ImageCollection('OREGONSTATE/PRISM/AN81m')
             .select(['tmean'])
             .filter(ee.Filter.date('1973-01-01', '2023-12-31')))

reduce_prism = create_reduce_region_function(
    geometry=point, reducer=ee.Reducer.first(), scale=1000, crs='EPSG:4326')

prism_stat_fc = (ee.FeatureCollection(prism_col.map(reduce_prism))
                 .filter(ee.Filter.notNull(prism_col.first().bandNames())))

prism_dict = fc_to_dict(prism_stat_fc).getInfo()
prism_df = pd.DataFrame(prism_dict)

prism_df = add_date_info(prism_df)
prism_df['Model'] = 'PRISM'
prism_df = prism_df.rename(columns={'tmean': 'Temp-mean'})

climate_df = pd.concat([prism_df, dcp_df], sort=True)

climate_df['Temp-mean'] = (climate_df['Temp-mean']*(9/5)) + 32

climate_df = filter_summer_months(climate_df)

climate_df = climate_df.drop(['DOY','Day','Month','millis'],axis=1)

summer_mean_df = climate_df.groupby(['Year']).mean(['Temp-mean']).reset_index()

In [ ]:
summer_mean_df.head(10)

In [ ]:
line = alt.Chart(climate_df).mark_line().encode(
    x='Year:O',
    y='median(Temp-mean):Q',
    color='Model')

band = alt.Chart(climate_df).mark_errorband(extent='iqr').encode(
    x='Year:O',
    y=alt.Y('Temp-mean:Q', title='Temperature (°F)'), color='Model')

(band + line).properties(width=900, height=300)